In [1]:
import pandas as pd
import load_data

In [2]:
questions, context = load_data.get_siqa_data()

Context: Tracy didn't go home that evening and resisted Riley's attacks.
Question: What does Tracy need to do before this?
Correct Label: 3


In [3]:
context_dict = {'context': context[:1000]}  
context_df = pd.DataFrame(context_dict) 
    
# write the DataFrame to a CSV file
# df.to_csv('SIQA_context_1000.csv', index=True, index_label='ID') 

In [11]:
# empty gpu
import gc
import torch

torch.cuda.empty_cache() 
gc.collect()

0

In [12]:
# !wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q6_K.gguf

In [13]:
!gpustat -cp

jupyter-notebook-cuda-65565644dd-zvg9q  Wed Oct 11 23:14:07 2023  515.65.01
[0] Tesla P100-SXM2-16GB | 34°C,   4 % |     0 / 16384 MB |


In [14]:
import llama_cpp
from llama_cpp import Llama

In [15]:
# model_path = "BIG-bench/llama-2-13b.Q4_0.gguf"
model_path = "BIG-bench/llama-2-13b-chat.Q6_K.gguf"

In [16]:
llm = Llama(model_path=model_path, n_gpu_layers=-1, n_ctx=4096, logits_all=True, verbose=False)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla P100-SXM2-16GB, compute capability 6.0
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from BIG-bench/llama-2-13b-chat.Q6_K.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q6_K     [  5120,  5120,     1,     1 ]
l

In [17]:
def get_llm_output(prompt):
  response = ""
  response = llm(prompt, temperature=0.8, repeat_penalty = 1.1, max_tokens=1000)['choices'][0]['text']
  # for token in llm.generate(prompt):
  #   response += token

  # pattern = r"<p>(.*?)</p>"
  # matches = re.search(pattern, response)

  # if matches:
  #     matched_text = matches.group(1)
  #     return matched_text
  return response

In [14]:
count = 0
template = """
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. Give 3 unique paraphrases. Change styles based on gender linked language.
A sample of word preferences for Male and Female style is given below, you can extrapolate your know 
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. Male : Paraphrased sentence\n2. Female :  Paraphrased sentence\n3. Ambiguous : Paraphrased sentence\n
Document: "{sentence}"
[/INST]
"""
# "Use the sentence given in brackets ({sentence}). Instruction: Paraphrase the sentence while preserving its meaning, in your text reponse give the paraphrased sentence enclosed in <sent></sent>"
# prompt.format(sentence="the sentence")
print("PROMPT - ", template)
for q, c in zip(questions, context):
  print("---------------")
  print("Sentence: ", c,"\n")
  prompt = template.format(sentence=c)
  print("Paraphrase: ", get_llm_output(prompt))
  count+=1
  if count==100:
    break

PROMPT -  
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. Give 3 unique paraphrases.
Change styles based on gender linked language.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV and gender as the sentence 
Format: 1. Male : Paraphrased sentence
2. Female :  Paraphrased sentence
3. Ambiguous : Paraphrased sentence

Document: "{sentence}"
[/INST]

---------------
Sentence:  Tracy didn't go home that evening and resisted Riley's attacks. 

Paraphrase:  Sure, I'd be happy to help! Here are three unique paraphrases of the given document based on gender-linked language:

1. Male: Tracy skipped going home that night and fought back against Riley's aggression.
2. Female: Tracy decided not to go home that evening and defended herself